In [1]:
from __future__ import unicode_literals
from textacy import fileio, preprocess, extract, Corpus
import numpy as np

In [2]:
summary_docs = fileio.read_file_lines('/home/SMRT-labeled/social-docs')
summary_meta = fileio.read_json_lines('/home/SMRT-labeled/social-meta')

In [3]:
text = [t for t in summary_docs]
meta = [m for m in summary_meta][0]

print(len(text), len(meta))

(1937, 1937)


In [4]:
for k in meta[0]:
    print k, meta[0][k]

Category smrt bus


In [5]:
#load lable data
label_key = u'Category'

label_list = map(lambda x:x[label_key], meta)

In [6]:
import collections

collections.Counter(label_list)

Counter({u'#ref!': 28,
         u'(corporate)': 1,
         u'competitor (bus)': 93,
         u'competitor (buses)': 1,
         u'competitor (taxi)': 33,
         u'competitor (taxis)': 1,
         u'competitor (train)': 61,
         u'competitor (trains)': 30,
         u'competitor bus': 7,
         u'competitor train)': 1,
         u'general mentions': 2,
         u'industry': 390,
         u'smrt': 1,
         u'smrt (corporate)': 88,
         u'smrt bus': 502,
         u'smrt buses': 5,
         u'smrt corporate': 1,
         u'smrt taxi': 68,
         u'smrt taxis': 4,
         u'smrt train': 280,
         u'smrt trains': 339,
         u'taxis': 1})

In [7]:
#re-label: step 1 -- remove 'forum' and text including 'Same as'
text = [x[0] for x in zip(text, label_list) if (x[1] != u'#ref!')]
label_list = [x for x in label_list if ((x != u'#ref!'))]

def hasContent(wds):
    if((wds != u'#ref!') and (len(wds) > 5)):
        return True
    else:
        return False

maskKeyword = map(hasContent, text)
text = [x[1] for x in zip(maskKeyword, text) if(x[0])]
label_list = [x[1] for x in zip(maskKeyword, label_list) if(x[0])]

print(len(text), len(label_list))

(1909, 1909)


In [8]:
collections.Counter(label_list)

Counter({u'(corporate)': 1,
         u'competitor (bus)': 93,
         u'competitor (buses)': 1,
         u'competitor (taxi)': 33,
         u'competitor (taxis)': 1,
         u'competitor (train)': 61,
         u'competitor (trains)': 30,
         u'competitor bus': 7,
         u'competitor train)': 1,
         u'general mentions': 2,
         u'industry': 390,
         u'smrt': 1,
         u'smrt (corporate)': 88,
         u'smrt bus': 502,
         u'smrt buses': 5,
         u'smrt corporate': 1,
         u'smrt taxi': 68,
         u'smrt taxis': 4,
         u'smrt train': 280,
         u'smrt trains': 339,
         u'taxis': 1})

In [9]:
def reLabel(x):
    if(x in [u'(corporate)', u'smrt (corporate)', u'smrt corporate', u'smrt', u'general mentions']):
        return u'SMRT Corporate'
    elif(x in [u'smrt bus', u'smrt buses']):
        return u'SMRT Buses'
    elif(x in [u'smrt train', u'smrt trains']):
        return u'SMRT Trains'
    elif(x in [u'smrt taxi', u'smrt taxis']):
        return u'SMRT Taxis'
    elif(x in [u'competitor (bus)', u'competitor (buses)', u'competitor bus']):
        return u'Buses'
    elif(x in [u'competitor (taxi)', u'competitor (taxis)', 'taxis']):
        return u'Taxis'
    elif(x in [u'competitor (train)', u'competitor (trains)', u'competitor train)']):
        return u'Trains'
    elif(x in [u'industry']):
        return u'Public Transport Industry'
    else:
        return u'Unknown'

label_list = map(reLabel, label_list)
collections.Counter(label_list)

Counter({u'Buses': 101,
         u'Public Transport Industry': 390,
         u'SMRT Buses': 507,
         u'SMRT Corporate': 93,
         u'SMRT Taxis': 72,
         u'SMRT Trains': 619,
         u'Taxis': 35,
         u'Trains': 92})

In [10]:
#load existing traditional media model to test
from sklearn.externals import joblib
#regression
msg_model = joblib.load('/home/pretrain-model/category-ann-reg.pkl')
id2term_itr = fileio.read_json('/home/pretrain-model/category-id2term-ann-reg')
label2id_itr = fileio.read_json('/home/pretrain-model/category-label2id-ann-reg')

id2term = [i for i in id2term_itr][0]
label2id = [i for i in label2id_itr][0]

In [11]:
label2id

{u'buses': 3,
 u'competitors': 5,
 u'overseas news': 1,
 u'public transport industry': 2,
 u'smrt buses': 6,
 u'smrt corporate': 4,
 u'smrt taxis': 8,
 u'smrt trains': 9,
 u'taxi': 0,
 u'trains': 7}

In [12]:
corpus_train = Corpus('en', texts = text)

In [13]:
#topic model
from textacy import vsm

terms_lists = (doc.to_terms_list(ngrams={2, 3}, named_entities=True, as_strings=True) for doc in corpus_train)
dtm_test, i2t_test = vsm.doc_term_matrix(terms_lists, weighting='tf', normalize=True, smooth_idf=True, min_df=3, max_df=0.95, max_n_terms=15000)

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [14]:
#build new index
test2train = dict()
for (i, j) in i2t_test.items():
    if j in id2term.values():
        train_i = id2term.keys()[id2term.values().index(j)]
        test2train[i]=train_i

In [15]:
test_feature = np.zeros((dtm_test.shape[0], len(id2term)))
dtm_test_array = dtm_test.toarray()

for (i, j) in test2train.items():
    test_feature[:, j] = dtm_test_array[:, i]

In [16]:
result = msg_model.predict(test_feature)
pred_index = [x.argmax() for x in result]

In [17]:
def Label2ID(x):
    if(x == u'Taxis'):
        return 0
    else:
        return label2id[x.lower()]

true_index = map(Label2ID, label_list)

In [21]:
from sklearn import metrics

print(metrics.classification_report(y_true=true_index, y_pred=pred_index))

             precision    recall  f1-score   support

          0       0.14      0.06      0.08        35
          2       0.58      0.06      0.10       390
          3       0.19      0.06      0.09       101
          4       0.13      0.61      0.21        93
          6       0.69      0.73      0.71       507
          7       0.09      0.18      0.12        92
          8       0.49      0.68      0.57        72
          9       0.66      0.59      0.63       619

avg / total       0.56      0.47      0.46      1909

